In [117]:
import sqlite3 
import psycopg2
import pandas as pd



dbname= 'ucykrkjj'
user = 'ucykrkjj'
password = 'qub1iGp24Aci4scgOyy7KUaYe6XVocke'
host = 'raja.db.elephantsql.com'

In [118]:
#establishing a connection and creating cursory object psycopg2
pg_conn = psycopg2.connect(dbname=dbname, user=user,
                           password=password, host=host)
pg_curs = pg_conn.cursor()

In [119]:
df = pd.read_csv('repos/tomfox1/SQL/module2-sql-for-analysis/titanic.csv')
df[df['Name'].str.contains('\'')]

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
28,1,3,Miss. Ellen O'Dwyer,female,24.0,0,0,7.8792
46,1,3,Miss. Bridget O'Driscoll,female,27.0,0,0,7.7500
185,1,3,Mrs. Thomas (Johanna Godfrey) O'Brien,female,26.0,1,0,15.5000
362,0,3,Mr. Thomas O'Brien,male,27.0,1,0,15.5000
456,0,3,Mr. Maurice O'Connor,male,17.0,0,0,7.7500
499,0,3,Miss. Bridget Mary O'Sullivan,female,21.0,0,0,7.6292
549,0,3,Mr. Timothy O'Brien,male,21.0,0,0,7.8292
626,0,3,Mr. Patrick D O'Connell,male,18.0,0,0,7.7333
650,1,3,Miss. Hanora O'Leary,female,16.0,0,0,7.8292


In [120]:
df['Name']=df['Name'].str.replace('\'','')
df[df['Name'].str.contains('\'')]

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare


In [121]:
#I was having problems with SQL, since the code would not run since some passengers had double quotes 

#Dakota's way by converting directly to list:

#df = pd.read_csv('repos/tomfox1/SQL/module2-sql-for-analysis/titanic.csv')
#df["Name"] = titanic["Name"].replace("'", '', regex=True)
#df = titanic.values.tolist()

In [122]:
#using sqlalchemy to create our cursory object (engine)
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [123]:
df.to_sql(name='titanic', con=engine)

In [124]:
#engine here is equivalent to our cursory object when using sqlite3
passengers = engine.execute('SELECT * FROM titanic').fetchall()

In [125]:
create_t = """
CREATE TABLE titanic (
        passenger_id SERIAL PRIMARY KEY,
        Survived INT,
        Pclass INT,
        Name VARCHAR(100) NOT NULL, 
        Sex VARCHAR(50) NOT NULL,
        Age INT, 
        Sibling_spouse INT, 
        Parents_children INT,
        Fare INT
);
"""

In [101]:
#creating our titanic table onto elephantSQL
pg_curs.execute(create_t)

In [102]:
#victory, checking by running our browser on ElephantSQL
for passenger in passengers:
    insert_passenger = """
        INSERT INTO titanic
        VALUES""" + str(passenger)
    pg_curs.execute(insert_passenger)

In [126]:
pg_conn.commit()

In [127]:
query = 'SELECT fare FROM "public"."titanic" LIMIT 10;'
pg_curs.execute(query) 

In [128]:
pg_curs.fetchall()

[(7,), (71,), (8,), (53,), (8,), (8,), (52,), (21,), (11,), (30,)]

In [ ]:
SELECT COUNT(survived) FROM "public"."tit"
WHERE "survived" = 1

SELECT COUNT(survived) FROM "public"."tit"
WHERE "survived" = 0

SELECT  pclass, COUNT(passenger_id)
FROM public.tit
GROUP BY pclass;

SELECT pclass, COUNT(survived) FROM "public"."tit"
WHERE survived=1
GROUP BY pclass

SELECT survived, AVG(age) FROM public.tit
GROUP BY survived

SELECT pclass, AVG(age) FROM public.tit
GROUP BY pclass

SELECT pclass, AVG(fare) FROM public.titanic
GROUP BY pclass

SELECT survived, AVG(fare) FROM public.titanic
GROUP BY survived

SELECT pclass, AVG(sibling_spouse) from public.titanic
GROUP BY pclass

SELECT survived, AVG(sibling_spouse) from public.titanic
GROUP BY survived

SELECT pclass, AVG(parents_children) from public.titanic
GROUP BY pclass

SELECT survived, AVG(parents_children) from public.titanic
GROUP BY survived

SELECT COUNT(DISTINCT(name)) from public.titanic

In [134]:
pg_curs.execute('SELECT COUNT(survived) FROM public.t WHERE survived = 1;')
pg_curs.fetchall()

[(342,)]

In [136]:
pg_curs.execute("""SELECT COUNT(survived) FROM public.t
WHERE survived = 0;""") 
pg_curs.fetchall()

[(545,)]

In [137]:
pg_curs.execute("""SELECT  pclass, COUNT(passenger_id)
FROM public.tit
GROUP BY pclass;""") 
pg_curs.fetchall()

[(1, 216), (3, 487), (2, 184)]

In [142]:
pg_curs.execute("""SELECT pclass, COUNT(survived) FROM "public"."t"
WHERE survived=1
GROUP BY pclass;""") 
pg_curs.fetchall() 

[(1, 136), (3, 119), (2, 87)]

In [143]:
pg_curs.execute("""SELECT survived, AVG(age) FROM public.tit
GROUP BY survived;""") 
pg_curs.fetchall() 

[(0, Decimal('30.1541284403669725')), (1, Decimal('28.4122807017543860'))]

In [144]:
pg_curs.execute("""SELECT pclass, AVG(fare) FROM public.titanic
GROUP BY pclass;""") 
pg_curs.fetchall() 

[(1, Decimal('84.2453703703703704')),
 (3, Decimal('13.7268993839835729')),
 (2, Decimal('20.7880434782608696'))]

In [145]:
pg_curs.execute("""SELECT survived, AVG(fare) FROM public.titanic
GROUP BY survived;""") 
pg_curs.fetchall() 

[(0, Decimal('22.2678899082568807')), (1, Decimal('48.4532163742690058'))]

In [146]:
pg_curs.execute("""SELECT pclass, AVG(sibling_spouse) from public.titanic
GROUP BY pclass;""") 
pg_curs.fetchall()

[(1, Decimal('0.41666666666666666667')),
 (3, Decimal('0.62012320328542094456')),
 (2, Decimal('0.40217391304347826087'))]

In [147]:
pg_curs.execute("""SELECT survived, AVG(sibling_spouse) from public.titanic
GROUP BY survived;""") 
pg_curs.fetchall()

[(0, Decimal('0.55779816513761467890')),
 (1, Decimal('0.47368421052631578947'))]

In [148]:
pg_curs.execute("""SELECT pclass, AVG(parents_children) from public.titanic
GROUP BY pclass;""") 
pg_curs.fetchall()

[(1, Decimal('0.35648148148148148148')),
 (3, Decimal('0.39630390143737166324')),
 (2, Decimal('0.38043478260869565217'))]

In [149]:
pg_curs.execute("""SELECT survived, AVG(parents_children) from public.titanic
GROUP BY survived;""") 
pg_curs.fetchall()

[(0, Decimal('0.33211009174311926606')),
 (1, Decimal('0.46491228070175438596'))]

In [150]:
#thus, no names are equal 
pg_curs.execute("""SELECT COUNT(DISTINCT(name)) from public.titanic;""") 
pg_curs.fetchall()

[(887,)]

In [ ]:
#missing bonus question 